<h1>APRENDIZAJE SUPERVISADO</h1>

En cada linea estara explicita, en forma de comentario "#", la función de esa fracción de codigo para el desarrollo de la actividad.

In [1]:
#Antes de iniciar instalar
#pip3 install pydotplus
#pip3 install sklearn
#pip3 install pandas
#pip3 install numpy
#pip3 install matplotlib

In [2]:
#Se importa todo el set de sklearn que utilizaremos( pandas, sklearn, numpy, joblib), 
#para la parte gráfica tenemos (IPython, pydotplus), usamos os para tener acceso a archivos del sistema.
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_graphviz  # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.preprocessing import StandardScaler
from sklearn.externals.six import StringIO  
import numpy as np
from IPython.display import Image  
import os
from joblib import dump, load 
import pydotplus
from io import StringIO
import random as rd
#For cross validation techniques
from sklearn import datasets, metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

/home/zirei/.local/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [3]:
#Es un método que crea datos de forma aleatoria para predecir el klout usando los modelos que se crearán más adelante.
def prediction_example(clf):
    # Each parameter has a range based on "datosLimpios.csv" training file
    # Ranges for simulation may vary if a new Training file is created
    Hour = rd.randrange(0,24)
    Day = rd.randrange(1,32)
    Reach_procentaje = rd.randrange(0,101)
    IsReshare = rd.randrange(0,2)
    RetweetCount = rd.randrange(0,26128)
    # Make new observation
    observation = [[Hour, Day, Reach_procentaje, IsReshare, RetweetCount]] 
    # Generate random values
    print("The generated values are: ", observation)
    # Predict observation's class (STARTING FROM 0 to 100%)    
    prediction = clf.predict(observation)
    print("class Klout predicted is:", prediction[0]) 

In [4]:
#Es un método que crea datos de forma aleatoria para predecir los likes que tendra una publicación en base a
#los modelos que se crearán más adelante.
def prediction_example2(clf):
    # Each parameter has a range based on "datosLimpios.csv" training file
    # Ranges for simulation may vary if a new Training file is created
    Hour = rd.randrange(0,24)
    Day = rd.randrange(1,32)
    IsReshare = rd.randrange(0,2)
    Reach = rd.randrange(0, 10342453)
    RetweetCount = rd.randrange(0,26128)
    # Make new observation
    observation = [[Hour, Day, IsReshare, Reach, RetweetCount]] 
    # Generate random values
    print("The generated values are: ", observation)
    # Predict observation's class (STARTING FROM 0 to 100%)    
    prediction = clf.predict(observation)
    print("You've got:", prediction[0], "Likes","\n") 

In [5]:
#El entrenamiento del modelo klout
def train_model():
    classifier_filename = 'classifier_model.joblib'
    feature_cols = ['Hour', 'Day', 'Reach_procentaje', 'IsReshare', 'RetweetCount']

    # Check if model file (.joblib extension) exist and use it as the classifier model
    model_exist = os.path.isfile(classifier_filename)

    if model_exist == False:  # Proceed to training
        col_names = ['Hour', 'Day', 'Reach_procentaje', 'IsReshare', 'RetweetCount', 'Klout']
        
        training = pd.read_csv("../datasets/datosLimpios.csv", header=0, names=col_names)  # load dataset
        
        # Split dataset in features and target variable
        x = training[feature_cols] # Features
        y = training.Klout # Target variable
        
        # Split dataset into training set and test set (70% training and 20% test)
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
       
        
        clf = DecisionTreeClassifier() # Create Decision Tree classifer object
        clf = clf.fit(x_train,y_train) # Train Decision Tree Classifer
        
        #Accurracy
        y_pred = clf.predict(x_test)
        #Validatión
        score = clf.score(x_train,y_train)
        preds = clf.predict(x_test)
    
        #Metrics
        calc_accuracy(clf, y_test, y_pred)
        validation(clf, score, preds, y_test, x_train, y_train)
                   
        # Graph tree data (the output is a "tree.png" image)
        #graph_data(clf, feature_cols)

        dump(clf, classifier_filename)  # Write model in a file (persistance)
    else:
        clf = load(classifier_filename)  # Load classification model from file 
        print("Running Klout classifier")
        
        # Call prediction_example function (classifying events instead of simulating them with the method
        # "output_events_randomly")
        for _ in range(10):
           prediction_example(clf)

In [6]:
#El entrenamiento del modelo de Likes
def train_model2():
    classifier_filename = 'classifier_model2.joblib'
    feature_cols = ['Hour', 'Day', 'IsReshare', 'Reach', 'RetweetCount']

    # Check if model file (.joblib extension) exist and use it as the classifier model
    model_exist = os.path.isfile(classifier_filename)

    if model_exist == False:  # Proceed to training
        col_names = ['Hour', 'Day', 'IsReshare', 'Reach', 'RetweetCount', 'Likes']
        
        training = pd.read_csv("../datasets/datosLimpios2.csv", header=0, names=col_names)  # load dataset
        
        # Split dataset in features and target variable
        x = training[feature_cols] # Features
        y = training.Likes # Target variable
        
        # Split dataset into training set and test set (70% training and 20% test)
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
       
        
        clf = DecisionTreeClassifier() # Create Decision Tree classifer object
        clf = clf.fit(x_train,y_train) # Train Decision Tree Classifer
        
        #Accurracy
        y_pred = clf.predict(x_test)
        #Validatión
        score = clf.score(x_train,y_train)
        preds = clf.predict(x_test)
    
        #Metrics
        calc_accuracy(clf, y_test, y_pred)
        validation(clf, score, preds, y_test, x_train, y_train)
                   
        # Graph tree data (the output is a "tree.png" image)
        #graph_data(clf, feature_cols)

        dump(clf, classifier_filename)  # Write model in a file (persistance)
    else:
        clf = load(classifier_filename)  # Load classification model from file 
        print("Running Likes classifier")
        
        # Call prediction_example function (classifying events instead of simulating them with the method
        # "output_events_randomly")
        for _ in range(10):
           prediction_example2(clf)

In [7]:
#Validación cruzada
def validation(clf, score, preds, y_test, x_train, y_train):
    # Calculate Accuracy. How often is the classifier correct?
        #Starting model validation metrics
                
        # To apply validation techniques
        kf = KFold(n_splits=5)
        
        print("Metrica del modelo", score)

        scores = cross_val_score(clf, x_train, y_train, cv=kf, scoring="accuracy")

        print("Metricas cross_validation", scores)

        print("Media de cross_validation", scores.mean())

        score_pred = metrics.accuracy_score(y_test, preds)

        print("Metrica en Test", score_pred)
        # End of model metrics

In [8]:
#Metricas de calidad, average weighted es para tomar el valor ponderado.
def calc_accuracy(clf, y_test, y_pred):
    # Calculate Accuracy. How often is the classifier correct?
    print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
    #print("Precision: ", metrics.precision_score(y_test, y_pred, average='weighted'))
    #print("Sensibilidad: ", metrics.recall_score(y_test, y_pred, average='weighted'))
    #print("Puntaje F1: ", metrics.f1_score(y_test, y_pred, average='weighted'))

In [9]:
#def graph_data(clf, feature_cols):
    #Creates an image (tree.png) that represents the shape of the classifier_tree
    #This is for illustrative purposes only (it uses a shorter training file than the current training file)
    #dot_data = StringIO()
    #export_graphviz(clf, out_file=dot_data,  
                    #filled=True, rounded=True,
                    #Name of the leaves of the tree
                    #special_characters=True, feature_names = feature_cols, class_names=['0','1','2','3','4','5'])
    #(graph, )= pydotplus.graph_from_dot_data(dot_data.getvalue())
    #graph.write_png('tree.png')
    #Image(graph.create_png())

In [10]:
#Klou statistics
train_model()

Accuracy: 0.105
Metrica del modelo 0.481625
Metricas cross_validation [0.1074375 0.103125  0.1084375 0.1050625 0.1043125]
Media de cross_validation 0.105675
Metrica en Test 0.105


In [ ]:
#Likes statistics
train_model2()

Accuracy: 0.99335
Metrica del modelo 1.0


In [ ]:
#Ejemplos simulados
train_model()
train_model2()